# Warlords Multi-Agent Toernooi
Dit notebook draait de Warlords-omgeving vanuit de Arcade Learning Environment (ALE) met 4 custom agenten.

Dit notebook is ontworpen om te draaien in Google Colab. Als je hem lokaal draait, moet je mogelijk meer libraries installeren en controleren of hun versies compatibel zijn.

## 1 Installeer libraries

Voer eerst de onderstaande codecell uit om de libraries te installeren.

In [ ]:
# Install the necessary libraries
!pip install pettingzoo[atari]
!pip install "autorom[accept-rom-license]"
!pip install --find-links dist/ --no-cache-dir AutoROM[accept-rom-license]

**Herstart nu je kernel**. Na het herstarten kun je direct doorgaan met de volgende codecel.

## 2 Importeer libraries en download Atari ROMs

Je krijgt een prompt om de AutoROM-overeenkomst te accepteren. Druk op "Y" wanneer je dit ziet.

In [ ]:
# Start AutoROM

!AutoROM

# Import libraries
from pettingzoo.atari import warlords_v3
from pettingzoo.utils import BaseParallelWrapper
import gymnasium as gym
import numpy as np
from collections import defaultdict, Counter
import importlib
import os
import imageio

## 3 Initialiseer agenten

In deze codecel importeren we de AI-agenten om Warlords te spelen. De bestanden met de agentklassen moeten zich in dezelfde map bevinden als dit notebook.

In [ ]:
# Import the agent classes
from agent1 import Agent1
from agent2 import Agent2
from agent3 import Agent3
from agent4 import Agent4

# Instantiate each agent (pass args if needed)
agent_instances = [
    Agent1(),
    Agent2(),
    Agent3(),
    Agent4()
]

agent_names = ['Agent1', 'Agent2', 'Agent3', 'Agent4']
scores = defaultdict(int)
wins = Counter()

## 4 Speel het spel

In deze sectie spelen de vier agenten Warlords. Aan het einde van elk spel wordt de score bijgehouden. De winnaar is de agent met de hoogste score.

In [ ]:
# Create environment
env = warlords_v3.env(render_mode="rgb_array")

# Prepare directory for videos
video_dir = "./warlords_videos"
os.makedirs(video_dir, exist_ok=True)

De volgende codecel speelt het spel.

In [ ]:
# Function to run one game and record video
def run_game(game_id):
    env.reset()

    # Map environment agents to their corresponding agent instances
    agent_mapping = {
        env.agents[i]: agent_instances[i]
        for i in range(len(env.agents))
    }

    # Map environment agents to their corresponding agent names
    agent_name_mapping = {
        env.agents[i]: agent_names[i]
        for i in range(len(env.agents))
    }

    # Reset scores
    for agent in agent_names:
        scores[agent] = 0

    done = False
    terminated = False
    truncated = False

    frames = []

    for agent in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()
        scores[agent_name_mapping[agent]] += reward

        if reward > 0:
            print(f"Agent {agent_name_mapping[agent]} won the game!")
            wins[agent_name_mapping[agent]] += 1

        # Use this to debug
        # print(f"Agent: {agent}, Name: {agent_name_mapping[agent]}, Reward: {reward}, Termination: {termination}, Truncation: {truncation}")

        if termination or truncation:
            action = None
        else:
            # this is where you would insert your policy
            agent_obj = agent_mapping[agent]
            action = agent_obj.act(observation)

        env.step(action)

        # Capture the rendered frame
        frame = env.render()
        frames.append(frame)
    env.close()

    # Save video using imageio
    video_path = os.path.join(video_dir, f"game_{game_id}.mp4")
    imageio.mimsave(video_path, frames, fps=15)

In [ ]:
# Run 10 games
for game in range(10):
    print(f"Running game {game + 1}...")
    run_game(game_id=game)

print("\nFinal Scores:")
for agent in agent_names:
    print(f"{agent}: Total Reward = {scores[agent]}, Wins = {wins[agent]}")

try:
    winner = wins.most_common(1)[0]
    print(f"Winner: {winner[0]} with {winner[1]} wins!")
except IndexError:
    print("No winners found.")

In [ ]:
# Display download links for videos
import glob
from IPython.display import FileLink, display
video_files = sorted(glob.glob(f"{video_dir}/*.mp4"))
print("\nDownload the recorded games:")
for file in video_files:
    display(FileLink(file))